In [12]:
import torch
from stgnn_model import STGNNModel

model = STGNNModel()
model.load_state_dict(torch.load("stgnn_stage2.pt", map_location="cpu"),strict=False)

bias = model.stgnn.gat.bias.detach().cpu().numpy()
print(bias)


[[ 0.          0.          0.        ]
 [-0.0070577   0.00705771  0.00705771]
 [-0.01242034  0.01242027  0.01242027]]


In [6]:
bias = model.stgnn.gat.bias.detach().cpu().numpy()
print(bias)


[[ 0.          0.          0.        ]
 [-0.02424086  0.23003557 -0.23088887]
 [-0.01894718 -0.22278465  0.22931455]]


In [14]:
import torch
from stgnn_model import STGNNModel

model = STGNNModel()
model.load_state_dict(torch.load("stgnn_reaction.pt", map_location="cpu"),strict= False)

bias = model.stgnn.gat.bias.detach().cpu().numpy()
print(bias)


[[ 0.          0.          0.        ]
 [-0.01587975  0.01587967  0.01587967]
 [-0.01696245  0.01696243  0.01696243]]


In [2]:
import numpy as np
import torch
import os

# =====================
# PATHS
# =====================
INPUT_181 = "reaction_inference/clip_0_listener_future_181.npy"
OUTPUT_621 = "reaction_inference/clip_0_listener_future_621.npy"
NORM_STATS = "/home/mudasir/Pawan/MPII/stacked_npy/checkpoints_tokenizer/norm_stats.pt"

# =====================
# Load data
# =====================
motion = np.load(INPUT_181).astype(np.float32)  # (2,150,181)

norm = torch.load(NORM_STATS, map_location="cpu")
mean = norm["mean"]
std = norm["std"]

# pad mean/std to 181
if mean.shape[0] < 181:
    pad = 181 - mean.shape[0]
    mean = torch.cat([mean, torch.zeros(pad)])
    std = torch.cat([std, torch.ones(pad)])

mean = mean.numpy()
std = std.numpy()

# =====================
# Denormalize
# =====================
motion_denorm = motion * std + mean   # (2,150,181)

# =====================
# Convert 181 → 621
# =====================
B, T, D = motion_denorm.shape

expression = motion_denorm[:, :, :177]
rotation = np.zeros((B, T, 3), dtype=np.float32)

shape = np.zeros((B, T, 156), dtype=np.float32)
texture = np.zeros((B, T, 251), dtype=np.float32)
lighting = np.zeros((B, T, 27), dtype=np.float32)
translation = np.tile(np.array([0, 0, 1], dtype=np.float32), (B, T, 1))
eye = np.zeros((B, T, 4), dtype=np.float32)

full_621 = np.concatenate([
    shape,
    expression,
    texture,
    lighting,
    rotation,
    translation,
    eye
], axis=2)

assert full_621.shape[2] == 621

np.save(OUTPUT_621, full_621)
print("✅ Saved 621-dim motion:", OUTPUT_621)


✅ Saved 621-dim motion: reaction_inference/clip_0_listener_future_621.npy


In [1]:
import numpy as np
import os

inp = "/home/mudasir/Pawan/MPII/simple_tranformer/STGNN/reaction_inference/recording13_clip20_listener_future_181.npy"
out_dir = "/home/mudasir/Pawan/MPII/simple_tranformer/STGNN/reaction_inference"
os.makedirs(out_dir, exist_ok=True)

data = np.load(inp)  # (2, 150, 621)

np.save(os.path.join(out_dir, "clip_0_listener1_621.npy"), data[0])
np.save(os.path.join(out_dir, "clip_0_listener2_621.npy"), data[1])

print("Saved:")
print("clip_0_listener1_621.npy", data[0].shape)
print("clip_0_listener2_621.npy", data[1].shape)


Saved:
clip_0_listener1_621.npy (150, 181)
clip_0_listener2_621.npy (150, 181)


In [3]:
import numpy as np
import os
import torch

# === INPUT / OUTPUT ===
input_path = "/home/mudasir/Pawan/MPII/simple_tranformer/STGNN/MPII_GroupReaction_Clean/motion/recording07/clip1/speaker.npy"
output_path = "/home/mudasir/Pawan/MPII/simple_tranformer/STGNN/reaction_inference/single_listener_0_future_621.npy"

# === NORMALIZATION STATS ===
norm_stats_path = "/home/mudasir/Pawan/MPII/stacked_npy/checkpoints_tokenizer/norm_stats.pt"

recon_norm = np.load(input_path).astype(np.float32)
T, D = recon_norm.shape

print(f"Loaded normalized prediction: {recon_norm.shape}")

norm = torch.load(norm_stats_path, map_location="cpu")
mean = norm["mean"]     # (177,)
std  = norm["std"]      # (177,)

# Pad to 181 dims (tokenizer input)
pad = 181 - mean.shape[0]
if pad > 0:
    mean = torch.cat([mean, torch.zeros(pad)])
    std  = torch.cat([std, torch.full((pad,), 1e-8)])

mean = mean.numpy()
std  = std.numpy()

print("Mean/std shape after padding:", mean.shape, std.shape)

recon_raw = recon_norm * std + mean

print("Denormalized motion shape:", recon_raw.shape)
print("Value range:", recon_raw.min(), recon_raw.max())

if D >= 177:
    expression = recon_raw[:, :177]
else:
    raise ValueError(f"Expected >=177 dims, got {D}")

print("Expression shape:", expression.shape)

rotation = np.zeros((T, 3), dtype=np.float32)

if D > 177:
    extra = recon_raw[:, 177:181]
    if extra.shape[1] >= 3:
        rotation[:, :3] = extra[:, :3]

print("Rotation shape:", rotation.shape)

shape = np.zeros((T, 156), dtype=np.float32)
texture = np.zeros((T, 251), dtype=np.float32)
lighting = np.zeros((T, 27), dtype=np.float32)

translation = np.tile(
    np.array([0.0, 0.0, 1.0], dtype=np.float32),
    (T, 1)
)

eye_rot = np.zeros((T, 4), dtype=np.float32)

full_621 = np.concatenate([
    shape,        # 0–155   (156)
    expression,   # 156–332 (177)
    texture,      # 333–583 (251)
    lighting,     # 584–610 (27)
    rotation,     # 611–613 (3)
    translation,  # 614–616 (3)
    eye_rot       # 617–620 (4)
], axis=1)

assert full_621.shape == (T, 621), f"❌ Wrong shape: {full_621.shape}"

print(f"✅ Final FaceVerse format: {full_621.shape}")
print("Final value range:", full_621.min(), full_621.max())

os.makedirs(os.path.dirname(output_path), exist_ok=True)
np.save(output_path, full_621)

print(f"💾 Saved: {output_path}")



Loaded normalized prediction: (102, 181)
Mean/std shape after padding: (181,) (181,)
Denormalized motion shape: (102, 181)
Value range: -0.293263 0.8504865
Expression shape: (102, 177)
Rotation shape: (102, 3)
✅ Final FaceVerse format: (102, 621)
Final value range: -0.293263 1.0
💾 Saved: /home/mudasir/Pawan/MPII/simple_tranformer/STGNN/reaction_inference/single_listener_0_future_621.npy


In [4]:
# make_video_moviepy.py
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip
import glob
import os

# Update these paths
frame_folder = "/home/mudasir/Pawan/FaceVerse_v4/renders/recording13_clip5_listener"  # your output folder
output_video = "/home/mudasir/Pawan/FaceVerse_v4/renders/recording13_clip5_listener.mp4"

# Get all frames in order
frames = sorted(glob.glob(os.path.join(frame_folder, "*frame_*.jpg")))
# or: frames = sorted(glob.glob(f"{frame_folder}/*.jpg"))

print(f"Found {len(frames)} frames → creating video...")

# Create video (30 FPS, high quality)
clip = ImageSequenceClip(frames, fps=30)
clip.write_videofile(
    output_video,
    codec="libx264",
    bitrate="5000k",
    preset="slow",
    ffmpeg_params=["-pix_fmt", "yuv420p"]  # Chrome/Safari compatible
)

print(f"DONE! Video saved: {output_video}")
print("Open it now — your face is ALIVE.")

Found 300 frames → creating video...
MoviePy - Building video /home/mudasir/Pawan/FaceVerse_v4/renders/recording13_clip5_listener.mp4.
MoviePy - Writing video /home/mudasir/Pawan/FaceVerse_v4/renders/recording13_clip5_listener.mp4



MoviePy - Done !
MoviePy - video ready /home/mudasir/Pawan/FaceVerse_v4/renders/recording13_clip5_listener.mp4
DONE! Video saved: /home/mudasir/Pawan/FaceVerse_v4/renders/recording13_clip5_listener.mp4
Open it now — your face is ALIVE.
